In [5]:
import os, cv2, re, random, shutil

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def copyFile(fileDir, tarDir, iteration = 4, persample = 80, sample_number = 3):
    # 1
    pathDir = os.listdir(fileDir)

    # 2
    pathDir.sort(key=natural_keys)

    print (len(pathDir))
    
    l_a = 0
    l_b = persample
    
    for i in range(iteration):
    
        print (l_a,l_b,sample_number, i)
        # 3
        sample = random.sample(pathDir[l_a:l_b], sample_number)
        print (sample)

        # 4
        for name in sample:
            #shutil.copyfile(fileDir + name, tarDir + name)
            shutil.move(fileDir + name, tarDir + name)
            
        l_a = l_a + persample
        l_b = l_b + persample
        

if __name__ == '__main__':
    TARGET_DIR = './train/'

    ORIGINAL_DIR = './test/'
    
    NB_CLASS = 2 
    NB_PER_CLASS = 80
    NB_SAMPLE = 1
    
    copyFile(ORIGINAL_DIR,TARGET_DIR, NB_CLASS, NB_PER_CLASS, NB_SAMPLE)

160
0 80 1 0
['Iris_0402.jpg']
80 160 1 1
['Pansy_1342.jpg']


In [5]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

# random example images
images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        ),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-90, 90), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 3),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
#                 iaa.OneOf([
#                     iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
#                     iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
#                     iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
#                 ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
#                 iaa.OneOf([
#                     iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
#                     iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
#                 ]),
#                 iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.ContrastNormalization((0.5, 2.0))
                    )
                ]),
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

print(images.shape)
images_aug = seq.augment_images(images)

(16, 128, 128, 3)


In [6]:
#Data Augmentation Test1
import os, cv2, re, random, shutil
from tqdm import tqdm
import numpy as np
from scipy import misc
import pandas as pd
from sklearn.model_selection import train_test_split

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

if __name__ == '__main__':
    SAMPLE_DIR = './train/'
    TARGET_DIR = './subsample/'
    
    if not os.path.exists(TARGET_DIR):
        os.makedirs(TARGET_DIR)

    subsample_amount = 1000

    sample_images_name = os.listdir(SAMPLE_DIR)
    sample_images = [SAMPLE_DIR + i for i in sample_images_name]  # use this for full dataset

    sample_images.sort(key=natural_keys)
    
    for img_path in tqdm(sample_images):

        # 导入图片
        img = misc.imread(img_path)
        img = misc.imresize(img, (128,128))
        
        img = np.expand_dims(img, axis=0)
        
        for i in tqdm(range(subsample_amount)):
            
            images_aug = seq.augment_images(img)
            
            img_save_path = TARGET_DIR + img_path[len(SAMPLE_DIR):-3] + str(i) + '.jpg'

#             print(img_save_path)
            
            img_save = images_aug[0, ...]

            misc.imsave(img_save_path, img_save)

  0%|          | 0/2 [00:00<?, ?it/s]/home/huiqy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/huiqy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.

  0%|          | 0/1000 [00:00<?, ?it/s]/home/huiqy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.

100%|██████████| 2/2 [00:13<00:00,  6.87s/it]83it/s]
